In [1]:
import json
import gzip
import tarfile
import numpy as np

def getIndex(l, indexes):
    return [l[i] for i in indexes]

def shuffle_data(index_path, data_path):
    index_data = []
    img_data = []
        
    with gzip.open(index_path, 'rt') as arc:
        index_data.extend(json.load(arc))
        print("index loaded")
        arc.close()
        
    intar = tarfile.open(data_path, "r:*")
    for i, member in enumerate(index_data):
        if i%10000 == 1:
            print("%2.0f%% complete (%d / %d)" % (i / len(index_data) * 100, i, len(index_data)))
        ti = intar.next()
        if ti.name != member['path']:
            print("ERROR: order doesn't match")
            break;
        f = intar.extractfile(ti)
        img_data.append(f)
    
    n = len(index_data)
    perm = np.random.permutation(n)
    
    index_data = getIndex(index_data, perm)
    img_data = getIndex(img_data, perm)
    
    with tarfile.open(data_path+".shuffled", "w|gz") as outtar:
        for i, f in enumerate(img_data):
            if i%1000 == 1:
                print("%2.0f%% complete (%d / %d)" % (i / len(index_data) * 100, i, len(index_data)))
            ti = tarfile.TarInfo(f.name)
            f.seek(0, 2)
            ti.size = f.tell()
            f.seek(0)
            outtar.addfile(ti, f)
        intar.close()
        outtar.close()
    
    with gzip.open(index_path+".shuffled") as arc:
        json.dump(index_data, arc, indent=4, sort_keys=True, separators=(',', ':'))
        arc.close()

In [2]:
shuffle_data('data/en/index.json.gz', 'data/en/data.tar.gz')
shuffle_data('data/ko/index.json.gz', 'data/ko/data.tar.gz')

index loaded
 0% complete (1 / 79200)
13% complete (10001 / 79200)
25% complete (20001 / 79200)
38% complete (30001 / 79200)
51% complete (40001 / 79200)
63% complete (50001 / 79200)
76% complete (60001 / 79200)
88% complete (70001 / 79200)
 0% complete (1 / 79200)
 1% complete (1001 / 79200)


KeyboardInterrupt: 